# Propositional logic

In [ ]:
data PropL = PVar String | PNot PropL | PropL `PAnd` PropL | PropL `POr` PropL deriving Eq

In [ ]:
instance Show PropL where
    show (PVar s) = s
    show (PNot p) = "~" ++ show p
    show (p `PAnd` q) = "(" ++ show p ++ " & " ++ show q ++ ")"
    show (p `POr` q) = "(" ++ show p ++ " | " ++ show q ++ ")"

In [ ]:
form1 = PVar "p1" `PAnd` PNot (PVar "p1" `POr` PVar "p3")

In [ ]:
form2 = (PVar "p3" `PAnd` PVar "p3") `POr` PNot (PNot (PVar "p1") `POr` PVar "p3")

## Number of operators

In [ ]:
opsNr :: PropL -> Int
opsNr (PVar _) = 0
opsNr (PNot p) = 1 + opsNr p
opsNr (PAnd p q) = 1 + opsNr p + opsNr q
opsNr (POr p q) = 1 + opsNr p + opsNr q

In [ ]:
opsNr form1

In [ ]:
depth :: PropL -> Int
depth (PVar _) = 0
depth (PNot p) = 1 + depth p
depth (PAnd p q) = 1 + max (depth p) (depth q)
depth (POr p q) = 1 + max (depth p) (depth q)

In [ ]:
depth form1

In [ ]:
depth form2

In [ ]:
gatherNames :: PropL -> [String]                  
gatherNames (PVar s) = [s]                        
gatherNames (PNot p) = gatherNames p                
gatherNames (PAnd p q) = gatherNames p ++ gatherNames q
gatherNames (POr p q) = gatherNames p ++ gatherNames q

In [ ]:
gatherNames form2

In [ ]:
import Data.List

propNames :: PropL -> [String]
propNames = sort . nub . gatherNames

In [ ]:
propNames form2

# Conversion to CNF

## de Morgan's (non recursive)

In [ ]:
dM1 :: PropL -> PropL
dM1 (PNot (p `POr` q)) = PNot p `PAnd` PNot q
dM1 p = p

In [ ]:
dM1 (PNot (PVar "p1" `POr` PVar "p2"))

In [ ]:
dM1 form2

In [ ]:
dM :: PropL -> PropL
dM (PNot (p `PAnd` q)) = PNot p `POr` PNot q
dM (PNot (p `POr` q)) = PNot p `PAnd` PNot q
dM p = p

In [ ]:
dM form2

## Recursive de Morgan's

In [ ]:
dMrec :: PropL -> PropL
dMrec (PNot (p `PAnd` q)) = PNot (dM p) `POr` PNot (dM q)
dMrec (PNot (p `POr` q)) = PNot (dM p) `PAnd` PNot (dM q)
dMrec (PNot p) = PNot (dM p)
dMrec (p `PAnd` q) = dM p `PAnd` dM q
dMrec (p `POr` q) = dM p `POr` dM q
dMrec (PVar p) = PVar p

In [ ]:
form2

In [ ]:
dMrec form2

## Recursive DNE

In [ ]:
dne :: PropL -> PropL
dne (PNot (PNot p)) = dne p
dne (PNot p) = PNot (dne p)
dne (p `PAnd` q) = dne p `PAnd` dne q 
dne (p `POr` q) = dne p `POr` dne q 
dne (PVar p) = PVar p

In [ ]:
toCNF :: PropL -> PropL
toCNF = dne . dMrec

In [ ]:
toCNF form2

## Distributive Law

In [ ]:
distLaw :: PropL -> PropL 
distLaw ((p `PAnd` q) `POr` (r `PAnd` s)) = (distLaw p `POr` distLaw r) `PAnd` (distLaw p `POr` distLaw s) `PAnd` (distLaw q `POr` distLaw r) `PAnd` (distLaw q `POr` distLaw s) -- double distributivity
distLaw (p `POr` (q `PAnd` r)) = (distLaw p `POr` distLaw q) `PAnd` (distLaw p `POr` r) --left dist
distLaw ((q `PAnd` r) `POr` p) = (distLaw q `POr` distLaw p) `PAnd` (distLaw r `POr` distLaw p) --right dist
distLaw (PNot p) = PNot (distLaw p)
distLaw (p `PAnd` q) = distLaw p `PAnd` distLaw q
distLaw (p `POr` q) = distLaw p `POr` distLaw q
distLaw (PVar p) = PVar p

In [ ]:
distLaw form2

In [ ]:
toCNF = distLaw . dne . dMrec

In [ ]:
toCNF form2

## Computing truth tables

In [ ]:
type VarAssignment = [(String,Bool)]
type TruthTable = [(VarAssignment,Bool)]

Our first step is to gather all of the variables used in a formula and generate all of the possible variable assignments.

In [ ]:
propNames form2

In [ ]:
{-# LANGUAGE ParallelListComp #-}

import Control.Monad (replicateM)

mkAssignments' :: [String] -> [VarAssignment]
mkAssignments' vs = [[(v,t) | v <- vs | t <- ts ] | ts <- replicateM (length vs) [True,False]]

mkAssignments'' vs = [ zip vs ts | ts <- replicateM (length vs) [True,False]]

In [ ]:
exAssigns = mkAssignments'' ["p", "q"]
exAssigns

In [ ]:
import Control.Monad (replicateM)

replicateM 3 [True,False]

In [ ]:
mkAssignments :: PropL -> [VarAssignment]
mkAssignments = mkAssignments' . propNames

In [ ]:
lookup :: (Eq a) => [(a,b)] -> a -> b
lookup [] _ = undefined
lookup ((key,val):ps) x = if key == x then val else lookup ps x

In [ ]:
mkAssignments form2

In [ ]:
exAssignments = mkAssignments form2 !! 0

In [ ]:
exAssignments

In [ ]:
lookup exAssignments "p1"

In [ ]:
interpretAtA :: VarAssignment -> PropL -> Bool
interpretAtA a (PVar v) = lookup a v
interpretAtA a (PNot p) = not (interpretAtA a p)
interpretAtA a (p `PAnd` q) = interpretAtA a p && interpretAtA a q
interpretAtA a (p `POr` q) = interpretAtA a p || interpretAtA a q

In [ ]:
interpretAtA exAssignments form2

In [ ]:
form2

In [ ]:
toTruthTable :: PropL -> [(VarAssignment,Bool)]
toTruthTable p = [(a, interpretAtA a p) | a <- mkAssignments p]

In [ ]:
toTruthTable form2

In [ ]:
interpretAtA [("p",True),("q",False)] (PVar "p" `PAnd` PVar "q")

In [ ]:
toTruthTable (PNot (PVar "p" `PAnd` PVar "q") `POr` PVar "r")

In [ ]:
formAnd = PVar "p1" `PAnd` PVar "p2"
exTable = toTruthTable formAnd

## Interpretation using a case expression

In [ ]:
interpretAtA a exp = case exp of
    (PVar p) -> lookup a p
    (PNot p) -> not (interpretAtA a p)
    (p `PAnd` q) -> interpretAtA a p && interpretAtA a q
    (p `POr` q) -> interpretAtA a p || interpretAtA a q

In [ ]:
gatherNames (PNot (PVar "p" `PAnd` PVar "q") `POr` PVar "r")